In [1]:
import sys
import pandas as pd
from Bio import SeqIO
from Bio import SearchIO
import os
import glob
from pathlib import Path
from Bio import AlignIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import numpy as np
import networkx as nx
from itertools import combinations
from pybedtools import BedTool

In [2]:
os.chdir('../../data')

In [3]:
#Import the fragment sequences around PRA genes
fna = list(SeqIO.parse('Au3_chr5.PR.fna', 'fasta'))

In [ ]:
#If the first three sequence is ATG then it is a start codon, add it to the list
start_codons = []
for seq in fna[:3]:
    if seq.seq[:3] == 'ATG':
        start_codons.append(seq)

In [4]:
#Find all the ORF  in the fragment sequences
orf_fwd = []
for f in range(0, len(fna), 1):
    for i in range(0, len(fna[f].seq), 1):
        if fna[f].seq[i:i+3] == 'ATG':
            for j in range(i+3, len(fna[f].seq), 3):
                if fna[f].seq[j:j+3] in ['TAA', 'TAG', 'TGA']:
                    #if the ORF is too short, skip
                    if j-i < 90:
                        break
                    else:
                        header = fna[f].id + '_' + str(i) + '_' + str(j) + '_fwd'
                        orf_fwd.append(SeqRecord(fna[f].seq[i:j+3], id=header, description='orf'))
                    break
orf_fwd_protein = []
for i in orf_fwd:
    orf_fwd_protein.append(SeqRecord(i.seq.translate(), id=i.id))
mfa_candidate_df_fwd = pd.DataFrame(columns=['aa', 'id', 'len'])
for i in orf_fwd_protein:
   pro_seq = i.seq
   #convert the pro_seq to string
   pro_seq = str(pro_seq)
   tmp = pd.DataFrame({'aa': pro_seq, 'id': i.id, 'len': len(pro_seq)}, index=[0])
   mfa_candidate_df_fwd = pd.concat([mfa_candidate_df_fwd, tmp], ignore_index=True)   

In [5]:
#Search for orf in the reverse strand
orf_rev = []
fna_rev = []
for f in range(0, len(fna), 1):
    fna_rev.append(SeqRecord(fna[f].reverse_complement().seq, id=fna[f].id, description='orf'))
for f in range(0, len(fna_rev), 1):
    for i in range(0, len(fna_rev[f].seq), 1):
        if fna_rev[f].seq[i:i+3] == 'ATG':
            for j in range(i+3, len(fna_rev[f].seq), 3):
                if fna_rev[f].seq[j:j+3] in ['TAA', 'TAG', 'TGA']:
                    #if the ORF is too short, skip
                    if j-i < 90:
                        break
                    else:
                        header = fna_rev[f].id + '_' + str(i) + '_' + str(j) + '_rev'
                        orf_rev.append(SeqRecord(fna_rev[f].seq[i:j+3], id=header, description='orf'))
                    break
orf_rev_protein = []
for i in orf_rev:
    orf_rev_protein.append(SeqRecord(i.seq.translate(), id=i.id))
mfa_candidate_df_rev = pd.DataFrame(columns=['aa', 'id', 'len'])
for i in orf_rev_protein:
   pro_seq = i.seq
   #convert the pro_seq to string
   pro_seq = str(pro_seq)
   tmp = pd.DataFrame({'aa': pro_seq, 'id': i.id, 'len': len(pro_seq)}, index=[0])
   mfa_candidate_df_rev = pd.concat([mfa_candidate_df_rev, tmp], ignore_index=True)   

In [6]:
mfa_candidate_df = pd.concat([mfa_candidate_df_fwd, mfa_candidate_df_rev], ignore_index=True)
mfa_candidate_df = mfa_candidate_df[mfa_candidate_df['aa'].str.contains('C[AVLI]{2}[A-Z]')]

In [7]:
mfa_candidate_mfa3 = mfa_candidate_df[mfa_candidate_df['aa'].str.contains('C[AVLI]{2}[A-Z]\*')]
mfa_candidate_mfa3 = mfa_candidate_mfa3[mfa_candidate_mfa3['aa'].str.contains('SHYC')]
mfa_candidate_mfa1 = mfa_candidate_df[mfa_candidate_df['aa'].str.contains('C[AVLI]{2}[A-Z]')]
mfa_candidate_mfa1 = mfa_candidate_mfa1[mfa_candidate_mfa1['aa'].str.contains('QWGN[A-Z]S')]

In [13]:
mfa_candidate_mfa1

,aa,id,len
50498,MNPTENPNNNDGKQWGNGSHICVIAREHPTEVEKQWGNGSHICIIA...,AU3_HapB_CHR05:20000000-30000000_2122646_21228...,53
50803,MNPNNNDGKQWGNGSHICVIAREPPTEVEKQWGNSSHICIIAQAPTAA*,AU3_HapB_CHR05:20000000-30000000_2197834_21979...,49
50959,MNPTENPKNNDGKQWANGSHICVIAREPPTEVEKQWGNGSHICIIA...,AU3_HapB_CHR05:20000000-30000000_2247080_22472...,53
51073,MNPTKNPNNNDGKQWGNGSHICVIAREPPTEVEKQWGNGSHICIIA...,AU3_HapB_CHR05:20000000-30000000_2288143_22882...,53
51231,MNPNNNNGKQWGNGSHICVIAREPPTKVKKQWGNGSHICIIAQAPTAA*,AU3_HapB_CHR05:20000000-30000000_2341032_23411...,49
...,...,...,...
154753,MNPTENPNNNDGKQWGNGSHICVIAKERPTEVEKQWGHGSHICIIA...,AU3_HapB_CHR05:20000000-30000000_7678541_76786...,53
154959,MNPTENPNNNDGKQWGNGSHICVIAREPPTEVEKQWGNGSHICIIA...,AU3_HapB_CHR05:20000000-30000000_7733285_77334...,53
155094,MNPTENPKNNDGKQWGNGSHICVIAREPPTEVEKQWGNGSHICIIA...,AU3_HapB_CHR05:20000000-30000000_7771961_77721...,53
155439,MNPTEKPNNNDGKQWGNSSHICVIARQPPTEVEKQWGNGSHICIIA...,AU3_HapB_CHR05:20000000-30000000_7858741_78588...,53


In [8]:
mfa_candidate_mfa2 = mfa_candidate_df[mfa_candidate_df['aa'].str.contains('CII[A-Z]\*')]
mfa_candidate_mfa2 = mfa_candidate_mfa2[mfa_candidate_mfa2['len'] <= 40]
mfa_candidate_mfa2

,aa,id,len
3928,MIWFTFLFWEQVLIGKAENFPIFSKCSCIIH*,AU3_HapA_CHR05:20000000-30000000_835253_835346...,32
112716,MIIKMMIWFNFLFWGQVLIGKAEFFPMFRKCICIIH*,AU3_HapA_CHR05:20000000-30000000_8087448_80875...,37
112717,MMIWFNFLFWGQVLIGKAEFFPMFRKCICIIH*,AU3_HapA_CHR05:20000000-30000000_8087460_80875...,33
112718,MIWFNFLFWGQVLIGKAEFFPMFRKCICIIH*,AU3_HapA_CHR05:20000000-30000000_8087463_80875...,32
113730,MIRKMMIWCTFLFWEQVFIGPAEFNPILRKFSCIIH*,AU3_HapA_CHR05:20000000-30000000_8247868_82479...,37
113731,MMIWCTFLFWEQVFIGPAEFNPILRKFSCIIH*,AU3_HapA_CHR05:20000000-30000000_8247880_82479...,33
113732,MIWCTFLFWEQVFIGPAEFNPILRKFSCIIH*,AU3_HapA_CHR05:20000000-30000000_8247883_82479...,32
114703,MISKMMIWFTFLFWEQVLIGKEEFFPMFRKCSCIIH*,AU3_HapA_CHR05:20000000-30000000_8448434_84485...,37
114704,MMIWFTFLFWEQVLIGKEEFFPMFRKCSCIIH*,AU3_HapA_CHR05:20000000-30000000_8448446_84485...,33
114705,MIWFTFLFWEQVLIGKEEFFPMFRKCSCIIH*,AU3_HapA_CHR05:20000000-30000000_8448449_84485...,32


In [9]:
mfa_candidate_mfa_filter = pd.concat([mfa_candidate_mfa1, mfa_candidate_mfa2, mfa_candidate_mfa3], ignore_index=True)
mfa_candidate_mfa_filter['chr'] = mfa_candidate_mfa_filter['id'].str.split(':').str[0]
mfa_candidate_mfa_filter['strand'] = mfa_candidate_mfa_filter['id'].str.split('_').str[5]

#If the strand is reverse, the start equals to 3000000 - end
mfa_candidate_mfa_filter.loc[mfa_candidate_mfa_filter['strand'] == 'rev', 'start'] = 30000000 - (mfa_candidate_mfa_filter['id'].str.split('_').str[4]).astype(int)-3
mfa_candidate_mfa_filter.loc[mfa_candidate_mfa_filter['strand'] == 'rev', 'end'] = 30000000 - (mfa_candidate_mfa_filter['id'].str.split('_').str[3]).astype(int)
mfa_candidate_mfa_filter.loc[mfa_candidate_mfa_filter['strand'] == 'fwd', 'start'] = 20000000 + (mfa_candidate_mfa_filter['id'].str.split('_').str[3]).astype(int)
mfa_candidate_mfa_filter.loc[mfa_candidate_mfa_filter['strand'] == 'fwd', 'end'] = 20000000 +(mfa_candidate_mfa_filter['id'].str.split('_').str[4]).astype(int)+3

mfa_candidate_mfa_filter = mfa_candidate_mfa_filter.sort_values(by=['chr', 'start', 'end'])
mfa_candidate_mfa_filter = mfa_candidate_mfa_filter.reset_index(drop=True)
#for fwd strand, group by chr, if end identical, only keep the longest one
mfa_candidate_mfa_filter_fwd = mfa_candidate_mfa_filter[mfa_candidate_mfa_filter['strand'] == 'fwd']
mfa_candidate_mfa_filter_fwd = mfa_candidate_mfa_filter_fwd.groupby(['chr', 'end'], as_index=False).apply(lambda x: x.loc[x['len'].idxmax()]).reset_index(drop=True)
mfa_candidate_mfa_filter_fwd = mfa_candidate_mfa_filter_fwd.reset_index(drop=True)
mfa_candidate_mfa_filter_rev = mfa_candidate_mfa_filter[mfa_candidate_mfa_filter['strand'] == 'rev']
mfa_candidate_mfa_filter_rev = mfa_candidate_mfa_filter_rev.groupby(['chr', 'start'], as_index=False).apply(lambda x: x.loc[x['len'].idxmax()]).reset_index(drop=True)
mfa_candidate_mfa_filter_rev = mfa_candidate_mfa_filter_rev.reset_index(drop=True)
mfa_candidate_mfa_filter = pd.concat([mfa_candidate_mfa_filter_fwd, mfa_candidate_mfa_filter_rev], ignore_index=True)
mfa_candidate_mfa_filter = mfa_candidate_mfa_filter.sort_values(by=['chr', 'start', 'end'])
mfa_candidate_mfa_filter = mfa_candidate_mfa_filter.reset_index(drop=True)
mfa_candidate_mfa_filter


/tmp/ipykernel_1414/520813089.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mfa_candidate_mfa_filter_fwd = mfa_candidate_mfa_filter_fwd.groupby(['chr', 'end'], as_index=False).apply(lambda x: x.loc[x['len'].idxmax()]).reset_index(drop=True)
/tmp/ipykernel_1414/520813089.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mfa_candidate_mfa_filter_rev = mfa_candidate_mfa_filter_rev.groupby(['chr', 'st

,aa,id,len,chr,strand,start,end
0,MIWFTFLFWEQVLIGKAENFPIFSKCSCIIH*,AU3_HapA_CHR05:20000000-30000000_835253_835346...,32,AU3_HapA_CHR05,fwd,20835253.0,20835349.0
1,MISKMMIWFTFLFWEQVLIGKEEFFPMFRKCSCIIH*,AU3_HapA_CHR05:20000000-30000000_8448434_84485...,37,AU3_HapA_CHR05,rev,21551455.0,21551566.0
2,MIRKMMIWCTFLFWEQVFIGPAEFNPILRKFSCIIH*,AU3_HapA_CHR05:20000000-30000000_8247868_82479...,37,AU3_HapA_CHR05,rev,21752021.0,21752132.0
3,MIIKMMIWFNFLFWGQVLIGKAEFFPMFRKCICIIH*,AU3_HapA_CHR05:20000000-30000000_8087448_80875...,37,AU3_HapA_CHR05,rev,21912441.0,21912552.0
4,MIRKMMIWFNFLFWEQVFNGKGEFFPMFGKCICIIC*,AU3_HapB_CHR05:20000000-30000000_8480371_84804...,37,AU3_HapB_CHR05,rev,21519518.0,21519629.0
...,...,...,...,...,...,...,...
176,MNPTENPKNNDGKQWGNGSHICVIAREPPTEVEKQWGNGSHICIIA...,AU3_HapB_CHR05:20000000-30000000_8521746_85219...,53,AU3_HapB_CHR05,fwd,28521746.0,28521905.0
177,MFTTPYTLPISTSASSVDSKEWGNGSHNCIIGREWGNGSNHCVIGR...,AU3_HapB_CHR05:20000000-30000000_1427238_14274...,80,AU3_HapB_CHR05,rev,28572522.0,28572762.0
178,MFTTPYTLPISTSASSVDSKEWGNGSHNCIIGREWGNGSNHCVIGR...,AU3_HapB_CHR05:20000000-30000000_8573648_85738...,80,AU3_HapB_CHR05,fwd,28573648.0,28573888.0
179,MFTTPYTLPIGTSASSVDSKEWGNGSHNCIIGREWGNGSHICVIGR...,AU3_HapB_CHR05:20000000-30000000_1023849_10240...,80,AU3_HapB_CHR05,rev,28975911.0,28976151.0


In [12]:
for record in orf_fwd:
    if record.id in mfa_candidate_mfa_filter['id'].tolist():
        print(record.format('fasta'))
        #save the record to a file
        with open('mfa_candidate.cds', 'a') as f:
            f.write(record.format('fasta'))
for record in orf_rev:
    if record.id in mfa_candidate_mfa_filter['id'].tolist():
        print(record.format('fasta'))
        #save the record to a file
        with open('mfa_candidate.cds', 'a') as f:
            f.write(record.format('fasta'))

>AU3_HapA_CHR05:20000000-30000000_835253_835346_fwd orf
ATGATTTGGTTTACTTTTCTGTTTTGGGAACAGGTGTTGATTGGAAAGGCAGAAAACTTT
CCAATTTTCAGCAAATGCAGTTGCATTATCCATTGA

>AU3_HapB_CHR05:20000000-30000000_2122646_2122802_fwd orf
ATGAACCCCACTGAGAACCCCAACAACAACGATGGCAAGCAATGGGGTAATGGCTCCCAC
ATCTGCGTCATCGCCAGGGAGCACCCCACCGAGGTCGAGAAGCAATGGGGTAACGGTTCC
CATATTTGCATCATTGCCCAAGCCCCAACCGCTGCCTAA

>AU3_HapB_CHR05:20000000-30000000_2197834_2197978_fwd orf
ATGAACCCCAACAACAACGATGGCAAGCAATGGGGTAATGGCTCCCACATCTGCGTCATC
GCCAGGGAGCCCCCCACCGAGGTCGAGAAGCAATGGGGTAACAGTTCCCATATTTGCATC
ATTGCCCAAGCCCCAACCGCTGCCTAA

>AU3_HapB_CHR05:20000000-30000000_2247080_2247236_fwd orf
ATGAACCCCACTGAGAACCCCAAAAACAACGATGGCAAGCAATGGGCTAATGGCTCCCAC
ATCTGCGTCATCGCCAGGGAGCCCCCCACCGAGGTCGAGAAGCAATGGGGTAACGGTTCC
CATATTTGCATCATTGCCCAAGCCCCAACCGCTGCCTAA

>AU3_HapB_CHR05:20000000-30000000_2288143_2288299_fwd orf
ATGAACCCCACTAAGAACCCCAACAACAACGATGGCAAGCAATGGGGTAATGGCTCCCAC
ATCTGCGTCATCGCCAGGGAGCCCCCCACCGAGGTCGAGAAGCAATGGGGTAACGGTTCC
CATATTTGCATCAT

In [112]:
mfa_candidate_mfa_filter['header'] = mfa_candidate_mfa_filter['chr'] + ':' + mfa_candidate_mfa_filter['start'].astype(str) + '-' + mfa_candidate_mfa_filter['end'].astype(str) + '_' + mfa_candidate_mfa_filter['strand']
#save the mfa peptide sequences to a fasta file
mfa_candidate_mfa_filter_seq = []
for i in range(0, len(mfa_candidate_mfa_filter), 1):
    header = str(mfa_candidate_mfa_filter['header'][i])
    mfa_candidate_mfa_filter_seq.append(SeqRecord(Seq(mfa_candidate_mfa_filter['aa'][i]), id=header, description=''))
SeqIO.write(mfa_candidate_mfa_filter_seq, 'mfa_candidate_mfa_filter.faa', 'fasta')

181

In [103]:
fna[0].seq[835253:835349].translate()

Seq('MIWFTFLFWEQVLIGKAENFPIFSKCSCIIH*')

In [95]:
fna[0]

SeqRecord(seq=Seq('GTCCCTCAGAATCCCTTGCAGAGGACCAAACAAATTTTTCTTCTTTGAGAGTGT...TAT'), id='AU3_HapA_CHR05:20000000-30000000', name='AU3_HapA_CHR05:20000000-30000000', description='AU3_HapA_CHR05:20000000-30000000', dbxrefs=[])

In [84]:
#Find the aa sequences have CAAX motif and likely to be mfa1
aliphatic_residues = ['A', 'V', 'I', 'L', 'M']
#test = mfa_candidate_df[mfa_candidate_df['aa'].str.contains('C[AVLI]{2}[A-Z]\*')]
test = mfa_candidate_df[mfa_candidate_df['aa'].str.contains('SHIC')]
mfa2=mfa_candidate_df[mfa_candidate_df['aa'].str.contains('C[AVLI]{2}[A-Z]\*')]
mfa2=mfa2[mfa2['aa'].str.contains('NHYC')]

In [47]:
mfa2=mfa_candidate_df[mfa_candidate_df['aa'].str.contains('C[AVLI]{2}[A-Z]\*')]

In [48]:
#Find the sequence from the list which contain 'NHYC'
mfa2=mfa2[mfa2['aa'].str.contains('[A-Z]HYC')]
mfa2=mfa2['aa'].tolist()
mfa2 = set(mfa2)
mfa_candidate_df[mfa_candidate_df['aa'].isin(mfa2)]


,aa,id,len
0,MFTTPYTLPIGTSASSVDSKEWGNGSHNCIIGREWGNGSHICVIGR...,AU3_HapB_CHR05:20000000-30000000_1023849_10240...,80
0,MFTTPYTLPISTSASSVDSKEWGNGSHNCIIGREWGNGSNHCVIGR...,AU3_HapB_CHR05:20000000-30000000_1427238_14274...,80


In [61]:
#mfa_fwd = mfa_candidate_df_fwd[mfa_candidate_df_fwd['aa'].str.contains('C[AVLI]{2}[A-Z]\*')]
#mfa_fwd = mfa_fwd[mfa_fwd['aa'].str.contains('SHIC')]
#mfa_fwd = mfa_candidate_df_fwd[mfa_candidate_df_fwd['len'] == 37]
#mfa_fwd = mfa_fwd[mfa_fwd['aa'].str.contains('C[AVLI]{2}[A-Z]\*')]
mfa_fwd = mfa_candidate_df_fwd[mfa_candidate_df_fwd['aa'].str.contains('C[AVLI]{2}[A-Z]\*')]
mfa_fwd = mfa_fwd[mfa_fwd['aa'].str.contains('NHYC')]
mfa_fwd

,aa,id,len


In [13]:
blast_bed['start'] = blast_bed['sstart'] -300
blast_bed['end'] = blast_bed['send'] +300